In [2]:
from roboverse.assets.shapenet_object_lists import GRASP_TRAIN_OBJECTS, GRASP_TEST_OBJECTS, OBJECT_ORIENTATION_INDS
from collections import defaultdict
import glob
import numpy as np
import roboverse.bullet as p
import matplotlib.pyplot as plt

pybullet build time: May 31 2022 06:54:08


In [2]:
base_dir = '/nfs/kun2/users/chet/robo_exp/orient_trajs/'

In [3]:
def success_rate(dataset_pattern):
    dataset = np.load(glob.glob(dataset_pattern)[0], allow_pickle=True)
    return np.mean([traj['rewards'][-1] > 0 for traj in dataset])

In [4]:
grasp_success = defaultdict(list)
rotate_grasp_success = defaultdict(list)
for obj_name in GRASP_TRAIN_OBJECTS + GRASP_TEST_OBJECTS:
    for orient_num in range(6):
        grasp_dataset_pattern = f'{base_dir}{obj_name}_orient_{orient_num}_grasp/*.npy'
        rotate_grasp_dataset_pattern = f'{base_dir}{obj_name}_orient_{orient_num}_rotate_grasp/*.npy'
        grasp_success[obj_name].append(success_rate(grasp_dataset_pattern))
        rotate_grasp_success[obj_name].append(success_rate(rotate_grasp_dataset_pattern))

In [5]:
for obj_name in GRASP_TRAIN_OBJECTS + GRASP_TEST_OBJECTS:
    print(obj_name)
    print(np.array(rotate_grasp_success[obj_name]), np.array(grasp_success[obj_name]))
    print(np.array(rotate_grasp_success[obj_name]) - np.array(grasp_success[obj_name]))

conic_cup
[0.   0.22 0.1  0.02 0.02 0.  ] [0.   0.72 0.36 0.02 0.   0.  ]
[ 0.   -0.5  -0.26  0.    0.02  0.  ]
fountain_vase
[0.08 0.48 0.26 0.1  0.14 0.3 ] [0.12 0.76 0.84 0.3  0.16 0.02]
[-0.04 -0.28 -0.58 -0.2  -0.02  0.28]
circular_table
[0.   0.44 0.3  0.42 0.06 0.4 ] [0.02 0.86 0.24 0.36 0.   0.3 ]
[-0.02 -0.42  0.06  0.06  0.06  0.1 ]
hex_deep_bowl
[0.38 0.04 0.12 0.6  0.26 0.26] [0.16 0.02 0.2  0.56 0.42 0.  ]
[ 0.22  0.02 -0.08  0.04 -0.16  0.26]
smushed_dumbbell
[0.   0.14 0.22 0.   0.02 0.24] [0.   0.02 0.1  0.   0.   0.6 ]
[ 0.    0.12  0.12  0.    0.02 -0.36]
square_prism_bin
[0.02 0.12 0.14 0.02 0.06 0.02] [0.   0.54 0.6  0.   0.02 0.  ]
[ 0.02 -0.42 -0.46  0.02  0.04  0.02]
narrow_tray
[0.12 0.46 0.1  0.06 0.2  0.3 ] [0.   0.04 0.2  0.   0.2  0.94]
[ 0.12  0.42 -0.1   0.06  0.   -0.64]
colunnade_top
[0.04 0.38 0.4  0.22 0.08 0.22] [0.16 0.82 0.94 0.68 0.08 0.14]
[-0.12 -0.44 -0.54 -0.46  0.    0.08]
stalagcite_chunk
[0.   0.22 0.22 0.   0.   0.26] [0.   0.82 0.92 0.   0

In [6]:
for obj_name in GRASP_TRAIN_OBJECTS + GRASP_TEST_OBJECTS:
    orient_ind = OBJECT_ORIENTATION_INDS[obj_name]
    print(obj_name, rotate_grasp_success[obj_name][orient_ind], grasp_success[obj_name][orient_ind])

conic_cup 0.22 0.72
fountain_vase 0.3 0.02
circular_table 0.3 0.24
hex_deep_bowl 0.38 0.16
smushed_dumbbell 0.14 0.02
square_prism_bin 0.14 0.6
narrow_tray 0.12 0.0
colunnade_top 0.22 0.14
stalagcite_chunk 0.26 0.0
bongo_drum_bowl 0.16 0.06
pacifier_vase 0.64 0.54
beehive_funnel 0.32 0.24
crooked_lid_trash_can 0.18 0.04
toilet_bowl 0.18 0.08
pepsi_bottle 0.28 0.0
tongue_chair 0.14 0.02
modern_canoe 0.38 0.0
pear_ringed_vase 0.16 0.0
short_handle_cup 0.32 0.12
bullet_vase 0.24 0.56
glass_half_gallon 0.22 0.54
flat_bottom_sack_vase 0.12 0.18
trapezoidal_bin 0.38 0.02
vintage_canoe 0.14 0.0
bathtub 0.14 0.0
flowery_half_donut 0.12 0.0
t_cup 0.2 0.2
cookie_circular_lidless_tin 0.14 0.0
box_sofa 0.16 0.0
two_layered_lampshade 0.12 0.0
conic_bin 0.36 0.18
jar 0.2 0.06
bunsen_burner 0.42 0.92
long_vase 0.12 0.0
ringed_cup_oversized_base 0.32 0.8
aero_cylinder 0.12 0.0
square_rod_embellishment 0.08 0.0
grill_trash_can 0.32 0.06
shed 0.08 0.24
sack_vase 0.18 0.16
two_handled_vase 0.3 0.0
thick_

In [11]:
obj_yaws = {}
for obj_name in GRASP_TRAIN_OBJECTS + GRASP_TEST_OBJECTS:
    orient_ind = OBJECT_ORIENTATION_INDS[obj_name]
    dataset_path = glob.glob(f'/nfs/kun2/users/chet/robo_exp/orient_trajs/{obj_name}_orient_{orient_ind}_rotate_grasp/*.npy')[0]
    dataset = np.load(dataset_path, allow_pickle=True)
    yaws = []
    for traj in dataset:
        if traj['rewards'][-1] == 0:
            continue
        for obs in traj['observations'][20:]:
            quat = obs['state'][3:7]
            yaws.append(p.quat_to_deg(quat)[-1])
    hist, bin_edges = np.histogram(yaws, bins=np.arange(-180, 185, 5))
    obj_yaws[obj_name] = np.argmax(hist) * 5 - 180
print(repr(obj_yaws))